# 케라스 NLP를 활용한 GPT 텍스트 생성 도전하기 튜토리얼 자료

**진행자:** [최태균](https://www.linkedin.com/in/taekyoon-choi/)<br>
**참고자료:** [GPT text generation with KerasNLP](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/text_generation_gpt.ipynb)

## 소개

이번 실습에서는 케라스 NLP를 활용하여 간단한 GPT 텍스트 생성 모델을 만들어 보고자 합니다.
GPT 모델은 Transformer 기반 모델로 여러분이 입력하신 프롬프트 텍스트를 가지고 생성을 하는 모델입니다.

이번 실습에서는 [simplebooks-92](https://arxiv.org/abs/1911.12391) 이라는 간단한 영문 텍스트 코퍼스를 가지고 텍스트 생성 모델을 학습 합니다. 
이 데이터의 경우 고빈도 어휘를 자주 사용하기 때문에 작은 모델을 학습 및 실습하는데 적합한 데이터로 알려져 있습니다.

실습할 GPT 모델은 여러분들께서 아시는 GPT-2나 GPT-3가 아닌 **미니어쳐 GPT 모델** 입니다. (참고: 
[Text generation with a miniature GPT](https://keras.io/examples/generative/text_generation_with_miniature_gpt/))
이 미니어쳐 GPT 모델을 KerasNLP 라이브러리를 통해 다음과 같은 경험을 얻으실 수 있습니다.

- 텍스트 전처리를 위한 토크나이저 생성 및 동작 방식 이해 
- GPT 텍스트 생성 모델 구성과 학습에 대한 이해
- 학습한 GPT 모델에 대한 텍스트 생성 동작 방식에 대한 이해

**시작하기 전에 주의할 점**: Colab에서 실습하시게 되면 세션 설정에 반드시 GPU runtime을 설정해주시기 바랍니다.

## 실습을 위한 설치

본 실습은 아래 기술한 `pip install`을 해야 원활하게 실습이 가능합니다.

In [ ]:
!pip install tensorflow==2.10.0
!pip install git+https://github.com/keras-team/keras-nlp.git --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 438 kB 46.0 MB/s 
     |████████████████████████████████| 1.7 MB 54.1 MB/s 
     |████████████████████████████████| 5.9 MB 49.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

In [ ]:
import os
import keras_nlp
import tensorflow as tf
from tensorflow import keras

In [ ]:
print(tf.__version__)
print(keras_nlp.__version__)

2.10.0
0.3.0


## Settings & hyperparameters

In [ ]:
# Data
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_TRAINING_SEQ_LEN = 450

# Model
EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000  # Limits parameters in model.

# Training
EPOCHS = 6

# Inference
NUM_TOKENS_TO_GENERATE = 10

## 데이터 가져오기

먼저 여러분들께서 학습할 데이터셋을 가져와 보도록 합시다. 
여러분들이 학습하게 될 SimpleBooks 데이터셋은 작은 어휘 사전 크기에 많은 토큰 수를 가지고 있는 데이터셋 입니다. 
데이터 토큰 사이즈는 약 30M 정도로 `WikiText-103`데이터(100M)에 비해 30% 적어 간단하게 학습을 하고 확인하기에 용이합니다.

In [ ]:
keras.utils.get_file(
    origin="https://dldata-public.s3.us-east-2.amazonaws.com/simplebooks.zip",
    extract=True,
)
dir = os.path.expanduser("~/.keras/datasets/simplebooks/")

282386239/282386239 [==============================] - 11s 0us/step


## 텍스트 파일 데이터 전처리 파이프라인 만들기

다운로드한 simplebooks 데이터를 확인해보면 텍스트로 이루어진 파일이라는 것을 확인해볼 수 있습니다.
이 데이터를 모델 학습 데이터로 만들기 위해 `tf.data`를 가지고 파일에서 읽어오는 전처리 파이프라인을 구성합니다.

먼저 우리가 학습할 데이터를 잠깐 살펴보면 아래와 같이 단순한 텍스트 파일로 구성되어있는걸 확인할 수 있습니다.

In [ ]:
!head -n20 ~/.keras/datasets/simplebooks/simplebooks-2-raw/train.txt

More Jataka Tales

By

Ellen C. Babbitt



I

The Girl Monkey And The String Of Pearls


One day the king went for a long walk in the woods. When he came back to his own garden, he sent for his family to come down to the lake for a swim.

When they were all ready to go into the water, the queen and her ladies left their jewels in charge of the servants, and then went down into the lake.

As the queen put her string of pearls away in a box, she was watched by a Girl Monkey who sat in the branches of a tree near-by. This Girl Monkey wanted to get the queen's string of pearls, so she sat still and watched, hoping that the servant in charge of the pearls would go to sleep.

At first the servant kept her eyes on the jewel-box. But by and by she began to nod, and then she fell fast asleep.


### `tf.data` 를 이용한 파일 데이터 전처리 하기

모델학습을 위해 학습 데이터를 `tf.data`로 데이터 전처리 파이프라인을 어떻게 구성하는지 살펴보겠습니다.

`tf.data.TextLineDataset` 클래스를 생성하면 입력한 경로를 통해 텍스트 데이터를 불러와 `Dataset` 객체를 생성합니다.


**참고**: [tf.data을 활용한 텍스트 데이터 파이프라인 가이드](https://www.tensorflow.org/guide/data#consuming_text_data)

In [ ]:
NUM_ITERS = 10

dataset = tf.data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
print(dataset)

<TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


`Dataset` 객체는 `iterable` 클래스이기 때문에 dataset 객체에 for 구문을 적용하여 데이터를 확인할 수 있습니다. 

In [ ]:
for i, element in enumerate(dataset):
    print(element)

    if i > NUM_ITERS:
        break

tf.Tensor(b"Dave Darrin's Second Year At Annapolis", shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'Or', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'Two Midshipmen As Naval Academy "Youngsters"', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'By', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'H. Irving Hancock', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)


`as_numpy_iterator` 함수를 사용하면 각 데이터 인스턴스의 값을 출력해볼 수 있습니다.

In [ ]:
for i, element in enumerate(dataset.as_numpy_iterator()):
    print(element)
    if i > NUM_ITERS:
        break

b"Dave Darrin's Second Year At Annapolis"
b''
b'Or'
b''
b'Two Midshipmen As Naval Academy "Youngsters"'
b''
b'By'
b''
b'H. Irving Hancock'
b''
b''
b''


`Dataset` 객체에 `filter` 함수를 호출하면 `Dataset`객체가 for 구문에서 순회할 때 `filter`에 정의한 로직이 동작하여 출력하게 됩니다.

여기서는 텍스트의 길이가 `MIN_TRAINING_SEQ_LEN`로 정의한 값보다 큰 텍스트만을 필터하여 출력하는 결과를 확인할 수 있습니다.

In [ ]:
filtered_dataset = dataset.filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)

for i, element in enumerate(filtered_dataset.as_numpy_iterator()):
    print(element)

    if i > NUM_ITERS:
        break

b"Readers of that preceding volume will recall how Dave Darrin and Dan Dalzell entered the United States Naval Academy, one appointed by a Congressman and the other by a United States Senator. Such readers will remember the difficult time that Dave and Dan had in getting through the work of the first hard, grinding year. They will also recall how Dave Darrin, when accused of treachery to his classmates, patiently bided his time until he, with the aid of some close friends, was able to demonstrate his innocence. Our readers will also remember how two evil-minded members of the then fourth class plotted to increase Damn's disgrace and to drive him out of the brigade; also how these two plotters, Midshipmen Henkel and Brimmer, were caught in their plotting and were themselves forced out of the brigade. Our readers know that before the end of the first year at the Naval Academy, Dave had fully reinstated himself in the esteem of his manly classmates, and how he quickly became the most popu

`Dataset` 객체는 `filter`와 같은 여러 전처리 함수들을 호출하면서 데이터 처리 파이프라인을 구성할 수 있습니다.

아래 예시는 `filter`함수와 `batch`함수를 같이 호출한 결과압니다.
`batch`함수는 설정하여 한 iteration에 설정한 batch size 만큼 출력될 수 있도록 합니다.

In [ ]:
batched_dataset = dataset.filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN).batch(2)

for i, element in enumerate(batched_dataset.as_numpy_iterator()):
    print(element)
    
    if i > NUM_ITERS:
        break

[b"Readers of that preceding volume will recall how Dave Darrin and Dan Dalzell entered the United States Naval Academy, one appointed by a Congressman and the other by a United States Senator. Such readers will remember the difficult time that Dave and Dan had in getting through the work of the first hard, grinding year. They will also recall how Dave Darrin, when accused of treachery to his classmates, patiently bided his time until he, with the aid of some close friends, was able to demonstrate his innocence. Our readers will also remember how two evil-minded members of the then fourth class plotted to increase Damn's disgrace and to drive him out of the brigade; also how these two plotters, Midshipmen Henkel and Brimmer, were caught in their plotting and were themselves forced out of the brigade. Our readers know that before the end of the first year at the Naval Academy, Dave had fully reinstated himself in the esteem of his manly classmates, and how he quickly became the most pop

위에서 소개한 방식대로 이제 학습데이터와 평가데이터를 위한 데이터 파이프라인을 구성해보도록 합니다.

In [ ]:
# 학습 데이터셋 파이프라인
raw_train_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

# 평가 데이터셋 파이프라인
raw_val_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/valid.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
)

위와 같이 데이터셋 파이프라인을 정의하면 다음과 같은 데이터 인스턴스를 확인할 수 있습니다.

In [ ]:
next(iter(raw_train_ds))

<tf.Tensor: shape=(64,), dtype=string, numpy=
array([b"I have only heard of those days. But I should have liked to have seen the bluff kind faces above the stiff stocks and scarlet coats, and the joyous smiles which shone upon them. I should have liked to have heard the quiet town ringing with such blithe laughter. Little jokes would cause the people to laugh, as little accidents would cause them to shake their heads. Sandy Hope's horse, for instance, lost a shoe while at the gallop, stumbled, and threw its rider, dislocating his shoulder, and breaking his arm. What a sensation the news created! It could scarcely have been greater even though Sandy's brains had been dashed out. Not only Sandy himself, but Sandy's kindred to the remotest degree, were deeply commiserated. The commanding officer sent his compliments every morning with inquiries after him. The troop doctor was besieged by anxious acquaintances. Sandy's comrades never ceased calling upon him, and sat for hours drinking beer

## 토크나이저 학습하기

먼저 텍스트를 토큰화(Tokenize)를 하기 위한 토크나이저를 만들어 봅시다.
여기서 토크나이저를 만드는 목적은 기존의 어휘 단어에 대해서 쪼개어 사전 토큰 수를 줄이고자 하는 목적이 있습니다.
이 토크나이저는 우리가 학습하고자 하는 학습 데이터셋으로 구성이 되고 `VOCAB_SIZE` 설정에 따라 토크나이저 사전 토큰 수도 정해집니다.
사전 토큰 수를 조절함에 따라 나중에 학습할 모델 성능에 영향을 줄 수 있습니다.
예를들어, 만약 너무 작은 사전 토큰 수로 정의를 하게 된다면 모델 학습을 하는 경우 Out-of-vocabulary(OOV)가 많이 발생하여 생성 퀄리티에 크게 영향을 받을 수 있을 겁니다.

토크나이저를 정의하면서 3 개의 토큰을 사전에 추가하고자 합니다.

- `"[PAD]"` 토큰은 학습할 텍스트 데이터에 길이를 고정하게 되었을 때 남는 토큰 위치에 추가를 해주는 토큰입니다. `"[PAD]"` 토큰을 이용하여 길이가 짧은 텍스트 데이터에 대해서 길이를 맞춰주는 역할을 합니다.
- `"[UNK]"` 토큰은 토큰화를 하는데 사전에 존재하지 않는 토큰이 나왔을 때 대신 채워주는 토큰입니다. 
- `"[BOS]"` 토큰은 문장을 시작하는 신호로 쓰이기 위한 토큰입니다. 학습 데이터에서는 텍스트가 시작되는 지점에 `"[BOS]"` 토큰을 둡니다.

이번 실습에서는 `keras_nlp.tokenizers`에 있는 `compute_word_piece_vocabulary` 함수를 이용하여 Wordpiece 토크나이저 사전을 생성해보겠습니다.


In [ ]:
# 토크나이저 사전을 학습합니다.
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

생성한 토크나이저 사전은 아래와 같이 확인할 수 있습니다.

In [ ]:
print(vocab[:30])

['[PAD]', '[UNK]', '[BOS]', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=']


## 토크나이저 만들기

앞서 만든 Wordpiece 토크나이저 사전을 가지고 토크나이저 객체를 만들어 보겠습니다.
토크나이저 객체는 `keras_nlp.tokenizers.WordPieceTokenizer` 클래스를 생성하여 만듭니다.
(`WordPieceTokenizer`는 `BERT`라는 모델에서 주로 사용하는 토크나이저입니다.)

`WordPieceTokenizer` 생성시 `sequence_length`와 `lowercase`를 설정함으로서 토큰화를 하는데 텍스트 길이를 제한하고 알파벳 텍스트의 경우 소문자로만 표기할 수 있도록 합니다.

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True
)

생성한 토크나이저를 가지고 토큰화를 하면 아래와 같은 결과로 나타납니다.

In [ ]:
print([tokenizer.id_to_token(t) for t in tokenizer.tokenize('hello world!').numpy().tolist()])

['he', '##ll', '##o', 'world', '!', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

## 인덱싱 파이프라인 추가하기

앞서 구현한 데이터 파이프라인에 토큰화 작업과 실제 모델 학습을 위한 입력 데이터와 정답 레이블을 추가해보도록 하겠습니다.


`keras_nlp.layers.StartEndPacker` 클래스는 입력 텍스트 데이터의 길이를 제한하고 문장 시작 토큰 `"[BOS]"`를 추가해주는 역할을 합니다.

In [ ]:
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)

생성한 `StartEndPacker`를 호출해보면 `"[BOS]"` 토큰 id `2` 가 추가된 결과를 확인해볼 수 있습니다.

In [ ]:
tokens = tokenizer('hello world')
start_packer(tokens)

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([   2,  103, 1520,  291,  394,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)>

이제 토크나이저와 `StartEndPacker` 처리를 거쳐서 모델 입력과 정답 레이블을 출력할 수 있는 함수를 구현해 보겠습니다.
`preprocess` 함수를 선언하여 토큰화를 하고 `"[BOS]"` 토큰을 추가하는 과정을 구현합니다.

In [ ]:
def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels

선언한 함수의 결과는 아래와 같습니다.

In [ ]:
preprocess("This is test sentence.")

(<tf.Tensor: shape=(128,), dtype=int32, numpy=
 array([   2,  139,  124,   56,  410,  375, 1121,   15,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0], dtype=int32)>,
 <tf.Tensor: shape=(128,), dtype=int32, numpy=
 array([ 139,  124,   56, 

이전에 생성한 `raw_train_ds`, `raw_val_ds` 객체에 `map` 함수를 적용하여 토크화 파이프라인을 추가합니다.
`map` 함수는 전처리하는 함수를 입력하여 전처리 동작을 할 수 있도록 합니다.
`num_parallel_calls` 설정은 `map`에서 처리하는 것을 여러 스레드로 처리할 수 있도록 설정합니다.

추가로 `prefetch` 함수는 현 시점에서 그 다음에 출력할 데이터를 미리 준비하는 역할로 데이터 처리 latency를 향상시키는 효과가 있습니다. 
주로 데이터셋 처리가 끝나는 지점에 `prefetch`함수를 선언합니다.

In [ ]:
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

## GPT 모델 만들기

본격적으로 학습할 GPT 모델을 Keras로 만들어봅시다.
만들 GPT 모델은 다음과 같은 layer들로 구성이 됩니다.

- Embedding layer: 입력 토큰 인덱스에 대해 토큰과 위치에 대한 임베딩을 출력합니다.
- Decoder layer: 입력 토큰들 뒤에 생성할 다음 토큰 임베딩을 출력합니다.
- Logit layer: 최종적으로 출력할 다음 토큰 인덱스를 출력합니다.

먼저 모델 입력을 구현해줍니다. 
Keras에서는 `keras.layers.Input`을 통해 모델의 입력을 정의합니다.

In [ ]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)

Embedding Layer를 구현합니다. 
여기서는 GPT 모델에서 활용한 토큰과 위치에 대한 임베딩을 같이 활용합니다.
`keras_nlp.layers.TokenAndPositionEmbedding` 클래스를 활용하면 간단하게 임베딩 레이어를 구현할 수 있습니다.

Embedding Layer는 앞서 구현한 입력 객체를 레이어 객체에 입력으로 넣어줍니다.

In [ ]:
# Embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)

임베딩 레이어에 이어서 Decoder Layer를 구현합니다.
Decoder Layer는 `keras_nlp.layers.TransformerDecoder`를 활용합니다.
Transformer 모델에는 Encoder 방식과 Decoder 방식으로 구분되는데 GPT에서는 Decoder만 활용됩니다.

Decoder Layer는 여러 개의 layer로 쌓을 수 있어 `NUM_LAYERS` 만큼 디코더 레이어를 쌓도록 구현되었습니다.

In [ ]:
for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x) 

Logit Layer의 경우 마지막 Decoder Layer에서 출력한 값을 그대로 Dense 레이어를 거치도록 합니다.

여기서는 `keras.layers.Dense` 레이어 클래스를 이용하여 Logit Layer를 구현합니다.

In [ ]:
outputs = keras.layers.Dense(VOCAB_SIZE)(x)

이렇게 모델 레이어에 대한 구현이 완료되면 `keras.Model`로 GPT 모델을 만들어줍니다.
이때 Model은 입력과 출력에 대한 객체만 생성시 입력으로 넣어주면 앞서 구현한 레이어들을 가진 모델 객체가 출력됩니다.

모델 학습을 위한 Loss 함수와 성능 메트릭 함수를 위해 `tf.keras.losses.SparseCategoricalCrossentropy`와 `keras_nlp.metrics.Perplexity`를 생성합니다.


`tf.keras.losses.SparseCategoricalCrossentropy`는 Cross Entropy 함수로 출력 클래스 수가 많은 경우에 활용합니다.
`keras_nlp.metrics.Perplexity`는 텍스트 생성 성능을 측정할 때 주로 사용되는 지표로 수치가 낮을 수록 더 나은 택스트 생성을 보입니다.

이렇게 Loss 함수와 메트릭 함수가 준비되면 `model.compile`을 통해 모델 학습할 준비를 마칠 수 있습니다.

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)

model.compile(optimizer="adam", loss=loss_fn, metrics=[perplexity])

이렇게 만들어진 GPT 모델을 어떻게 구성되어있는지 summary로 확인할 수 있습니다.
모델 파라메터 사이즈를 확인하면 Embedding Layer와 Logit Layer가 굉장히 크다는 것을 알 수 있는데요. 반면 Decoder Layer는 레이어 수가 많아도 Embedding Layer에 비해 그리 크진 않습니다. 이렇게 토큰 사전 크기에 따라 파라메터 수가 얼마나 커질 수 있는지 보여줍니다.

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        1312768   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 dense (Dense)               (None, None, 5000)        128500

## 모델 학습

이제 지금까지 만든 모델을 `fit()`함수를 통해 학습해보도록 합시다.

In [ ]:
model.fit(train_ds, validation_data=val_ds, verbose=2, epochs=EPOCHS)

Epoch 1/6
3169/3169 - 238s - loss: 4.5088 - perplexity: 96.2405 - val_loss: 4.0023 - val_perplexity: 62.5531 - 238s/epoch - 75ms/step
Epoch 2/6
3169/3169 - 229s - loss: 4.0025 - perplexity: 57.6544 - val_loss: 3.8421 - val_perplexity: 52.8804 - 229s/epoch - 72ms/step
Epoch 3/6
3169/3169 - 233s - loss: 3.8910 - perplexity: 51.4962 - val_loss: 3.8000 - val_perplexity: 50.4783 - 233s/epoch - 74ms/step
Epoch 4/6
3169/3169 - 234s - loss: 3.8280 - perplexity: 48.3152 - val_loss: 3.7428 - val_perplexity: 47.8048 - 234s/epoch - 74ms/step
Epoch 5/6
3169/3169 - 232s - loss: 3.7847 - perplexity: 46.2416 - val_loss: 3.7241 - val_perplexity: 46.9146 - 232s/epoch - 73ms/step
Epoch 6/6
3169/3169 - 231s - loss: 3.7534 - perplexity: 44.7984 - val_loss: 3.7239 - val_perplexity: 46.7429 - 231s/epoch - 73ms/step


## 텍스트 생성 인퍼런스

이제 학습한 모델을 가지고 텍스트 생성을 해보도록 합시다.
데이터 전처리를 하면서 `"[BOS]"` 토큰에 대해서 이야기 했었는데요.
이 토큰을 입력 프롬프트로 텍스트를 생성해보도록 하겠습니다.


In [ ]:
prompt_tokens = tf.convert_to_tensor([tokenizer.token_to_id("[BOS]")])

텍스트 생성을 할 때는 `keras_nlp.utils`에 있는 모듈을 사용하고자 합니다.
`keras_nlp.utils`을 사용하여 텍스트 생성을 하기 위해서 `token_logits_fn()` 함수를 래퍼(wrapper)로 구성합니다. (이 부분은 뒤에서 활용하는 방법에서 다시 설명하도록 하겠습니다.)
이 함수는 GPT 모델 인퍼런스를 한 후 PAD 토큰을 제거하고 생성하는 그 다음 토큰만 출력합니다.

In [ ]:
def token_logits_fn(inputs):
    cur_len = inputs.shape[1]
    output = model(inputs)
    return output[:, cur_len - 1, :]  # return next token logits


사실 텍스트를 생성하기 위해 사실 생성을 하기 위한 여러 방법들을 구현해야합니다.
다행이도, `keras_nlp.utils`에서 텍스트 생성에 필요한 방법을 모두 만들어뒀습니다.
이제 어떻게 활용할 수 있는지 살펴보도록 합시다.

### 그리디 서치

그리디 서치는 가장 기본적인 텍스트 생성 방법입니다.
이 방법은 모델에서 출력한 토큰 확률 중 가장 높은 확률을 가진 것을 가지고 출력합니다.

In [ ]:
output_tokens = keras_nlp.utils.greedy_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Greedy search generated text: \n{txt}\n")

Greedy search generated text: 
b'[BOS] " i \' m not going to be a'



그리디 서치와 같은 경우는 텍스트 생성을 하면서 금방 어휘가 반복되는 문제가 있습니다.

이러한 문제를 출력된 토큰의 확률을 이용하여 해결하는 방법이 있는데 이 내용은 뒤에서 다시 살펴보도록 하겠습니다.

### 빔서치

빔서치는 `beam_search`에 정의한 `num_beams` 수 만큼의 토큰 시퀀스의 확률들을 조합해서 가장 가능한 확률로 그 다음 토큰을 생성해냅니다.
많은 `num_beams`를 설정하면 그만한 길이에 대한 토큰에 대한 확률을 저장해야하는 문제가 있지만 그리디 서치에 비해 효과적입니다.

**주의!:** 만약에 `num_beams`이 1이면 그리디 서치와 같습니다.

In [ ]:
output_tokens = keras_nlp.utils.beam_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    num_beams=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Beam search generated text: \n{txt}\n")

Beam search generated text: 
b'[BOS] " i don \' t want to tell you'



### 랜덤 서치

랜덤 서치는 말 그대로 토큰을 랜덤으로 출력하는 방식인데, 여기서는 생성된 토큰의 확률을 토대로 랜덤 출력을 하는 방식입니다.

In [ ]:
output_tokens = keras_nlp.utils.random_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Random search generated text: \n{txt}\n")

Random search generated text: 
b'[BOS] " tumbling ! i guess , of course'



이렇게 랜덤 서치를 활용하면 단어가 반복하면서 생성되지 않는 것을 확인할 수 있습니다.
이와 같은 방식으로 조금 더 좋은 택스트 생성을 할 수 있는 Top-k 서치를 살펴보겠습니다.

### Top-K 서치

위에 소개한 랜덤 서치와 비슷한 방법입니다. 
단, 차이점은 토큰 확률에서 Top-K 개만을 가지고 랜덤 서치를 하는 방식입니다.

이렇게 하는 이유는 가장 낮은 확률의 토큰이 나오면 맥락에 맞지 않는 토큰들이 생성될 가능성이 있기 때문입니다.
Top-K 서치를 사용하게 되면 적어도 랜덤 서치에 비해서 나은 택스트 생성 결과를 확인할 수 있습니다.

In [ ]:
output_tokens = keras_nlp.utils.top_k_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    k=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-K search generated text: \n{txt}\n")

Top-K search generated text: 
b'[BOS] " i can \' t say that it \''



### Top-P 서치

Top-K 서치의 경우 분명히 좋은 텍스트 생성을 하지만, 적절한 K 값을 찾는 것이 어렵습니다.
만약에 k=5로 했을 경우 5번째로 높은 토큰의 확률이 0.3일 경우도 있고 이보다 작은 0.1일 경우도 있습니다. 
다시말해 생성하는 토큰의 확률 분포에 따라 낮은 확률을 가진 토큰이 생성이 될 수도 있습니다.

이러한 문제를 토큰의 확률이 높은 순서가 아니라 일정 확률이 높은순에서 합한 누적확률을 기준으로 샘플링을 할 수 있는데 이런 방법이 Top-P 서치입니다.
Top-P 서치를 이용하여 P값을 0.8로 설정한다면 Top-1에서부터 높은 확률을 순차적으로 누적했을 때 0.8이 되는 토큰들만을 가지고 샘플링을 하는 방식입니다.
이 방식은 지금까지 가장 효과적인 택스트 생성 방식으로 알려져 있습니다.

In [ ]:
output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.5,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search generated text: \n{txt}\n")

Top-P search generated text: 
b'[BOS] when the three had left the peak'



### Callback을 이용하여 테스트에서 텍스트 생성 결과 확인해보기

Keras에서는 Callback 클래스를 이용하여 중간 평가를 해보거나 결과를 생성해볼 수 있습니다.
아래 코드에서는 간단한 Top-K 방식의 택스트 생성 Callback 클래스를 구현하여 학습에 어떻게 적용할 수 있는지 보여줍니다.

In [ ]:

class TopKTextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model using top-k."""

    def __init__(self, k):
        self.k = k

    def on_epoch_end(self, epoch, logs=None):
        output_tokens = keras_nlp.utils.top_k_search(
            token_logits_fn,
            prompt_tokens,
            max_length=NUM_TOKENS_TO_GENERATE,
            k=self.k,
            from_logits=True,
        )
        txt = tokenizer.detokenize(output_tokens)
        print(f"Top-K search generated text: \n{txt}\n")


text_generation_callback = TopKTextGenerator(k=10)
# Dummy training loop to demonstrate callback.
model.fit(train_ds.take(1), verbose=2, epochs=2, callbacks=[text_generation_callback])

Epoch 1/2
Top-K search generated text: 
b'[BOS] " i will tell you something about the world'

1/1 - 5s - loss: 3.9148 - perplexity: 51.8893 - 5s/epoch - 5s/step
Epoch 2/2
Top-K search generated text: 
b'[BOS] " you must remember my mother , " he'

1/1 - 5s - loss: 3.7833 - perplexity: 47.9053 - 5s/epoch - 5s/step


## 마무리

이렇게 해서 KerasNLP를 가지고 토크나이저와 미니어쳐 GPT를 만들어보고,
GPT 모델을 가지고 어떻게 텍스트를 생성하는지 까지 알아봤습니다.

모델에 대해서 보다 더 자세하게 보고싶은 분은 아래 두 논문을 참고해주시기 바랍니다.

- Attention Is All You Need [Vaswani et al., 2017](https://arxiv.org/abs/1706.03762)
- GPT-3 Paper [Brown et al., 2020](https://arxiv.org/abs/2005.14165)